<a href="https://colab.research.google.com/github/aa-jesse/Machine-Learning-Projects/blob/main/Credit_Card_Fraud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the Dependencies 

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
# import the dataset into a pandas dataframe
credit_card_info = pd.read_csv('/content/creditcard.csv')

In [5]:
credit_card_info.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [6]:
#check for missing values and the datatype for dataset
credit_card_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105067 entries, 0 to 105066
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    105067 non-null  int64  
 1   V1      105067 non-null  float64
 2   V2      105067 non-null  float64
 3   V3      105067 non-null  float64
 4   V4      105067 non-null  float64
 5   V5      105067 non-null  float64
 6   V6      105067 non-null  float64
 7   V7      105067 non-null  float64
 8   V8      105067 non-null  float64
 9   V9      105067 non-null  float64
 10  V10     105067 non-null  float64
 11  V11     105067 non-null  float64
 12  V12     105067 non-null  float64
 13  V13     105067 non-null  float64
 14  V14     105067 non-null  float64
 15  V15     105067 non-null  float64
 16  V16     105067 non-null  float64
 17  V17     105067 non-null  float64
 18  V18     105067 non-null  float64
 19  V19     105067 non-null  float64
 20  V20     105067 non-null  float64
 21  V21     10

In [7]:
#how to checl out missing values
credit_card_info.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [8]:
#distribution of both real and fraudulent transaction
#0 = real transaction
#1 = fraudulent transaction
credit_card_info['Class'].value_counts()

0.0    104834
1.0       232
Name: Class, dtype: int64

In [9]:
#seperate the dataframe into real and fraud transaction
legit = credit_card_info[credit_card_info.Class == 0]
Fraud = credit_card_info[credit_card_info.Class == 1]

In [10]:
print(legit.shape)
print(Fraud.shape)

(104834, 31)
(232, 31)


In [11]:
#stats about the real transactions
legit.Amount.describe()

count    104834.000000
mean         96.584297
std         262.333583
min           0.000000
25%           7.190000
50%          25.540000
75%          87.440000
max       19656.530000
Name: Amount, dtype: float64

In [13]:
#stats about the fraud transactions
Fraud.Amount.describe()

count     232.000000
mean      115.520474
std       253.721393
min         0.000000
25%         1.000000
50%         7.595000
75%        99.990000
max      1809.680000
Name: Amount, dtype: float64

In [14]:
#compare mean of varoius classes
credit_card_info.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,43692.851508,-0.245296,-0.039251,0.702486,0.146715,-0.274535,0.102607,-0.102664,0.053767,-0.044415,...,0.043860,-0.032987,-0.104940,-0.037011,0.010118,0.133410,0.025912,0.000488,0.001663,96.584297
1.0,37898.655172,-6.138432,4.280104,-7.850475,4.800030,-4.349687,-1.566011,-6.505922,1.640373,-2.788880,...,0.237631,1.386881,-0.351377,-0.114210,-0.118153,0.208771,0.094034,0.536235,0.061592,115.520474


Under-Sampling

In [15]:
#create new dataset with equal distribution of both fraud and legi transaction

legit_sample = legit.sample(n=232)
legit_sample.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
78469,57543,0.190815,-1.904716,-0.247482,0.644765,-1.058037,-0.298803,0.591606,-0.244797,0.573678,...,0.001829,-1.117827,-0.469945,-0.029399,0.081119,0.811501,-0.173564,0.100095,574.84,0.0
44278,41874,1.170689,0.155342,0.391393,1.398592,-0.221767,-0.202097,0.004314,0.035455,0.423721,...,-0.113927,-0.126234,-0.048466,0.070745,0.634873,-0.309517,0.034801,0.013572,5.00,0.0
36707,38654,1.265302,0.568235,-0.612150,1.985265,1.056360,0.467422,0.438874,-0.017196,-1.159272,...,-0.066564,-0.300899,-0.346935,-1.406394,0.913364,0.156997,-0.051615,-0.014403,25.03,0.0
101109,67727,1.298471,-0.226696,0.171038,-0.916361,-0.556145,-0.799933,-0.077636,-0.171860,1.624532,...,-0.279448,-0.510848,-0.075898,-0.105880,0.701276,-0.838111,0.070766,0.022204,8.62,0.0
50709,44610,-0.302386,1.119180,1.070261,-0.064141,0.291331,-0.677333,0.699144,-0.038867,-0.358379,...,-0.299384,-0.787907,-0.082553,-0.204081,-0.089096,0.105203,0.245147,0.092807,2.69,0.0


In [17]:
legit_sample.shape

(232, 31)

Concatenate 2 dataframes

In [19]:
new_credit_card_info = pd.concat([legit_sample, Fraud], axis=0)
new_credit_card_info.head()
new_credit_card_info.shape

(464, 31)

In [22]:
new_credit_card_info['Class'].value_counts()

0.0    232
1.0    232
Name: Class, dtype: int64

In [23]:
new_credit_card_info.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,41600.577586,-0.216717,0.121402,0.729042,0.172179,-0.244691,0.055577,-0.009062,-0.028505,-0.022954,...,0.083172,-0.082275,-0.133239,-0.056195,-0.021148,0.119038,0.025250,-0.009806,0.000237,92.590474
1.0,37898.655172,-6.138432,4.280104,-7.850475,4.800030,-4.349687,-1.566011,-6.505922,1.640373,-2.788880,...,0.237631,1.386881,-0.351377,-0.114210,-0.118153,0.208771,0.094034,0.536235,0.061592,115.520474


In [26]:
#spliting data into featues and targets
X = new_credit_card_info.drop(columns ='Class', axis=1)
Y = new_credit_card_info['Class']

In [28]:
print(Y)

78469     0.0
44278     0.0
36707     0.0
101109    0.0
50709     0.0
         ... 
102443    1.0
102444    1.0
102445    1.0
102446    1.0
102782    1.0
Name: Class, Length: 464, dtype: float64


In [32]:
#split into traininf and test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=3)

In [33]:
print(X.shape, X_train.shape, X_test.shape)

(464, 30) (371, 30) (93, 30)


Model Training-Logistic Regression

In [39]:
model = LogisticRegression(solver='lbfgs', max_iter=200)

In [40]:
#train the model with training data
model.fit(X_train, Y_train)

LogisticRegression(max_iter=200)

Model Evaluation - Accuracy Score

In [44]:
#accuracy on training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print('Accuracy on Training Data: ', training_data_accuracy)

Accuracy on Training Data:  0.967654986522911


In [46]:
#accuracy on test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print('Accuracy on Test Data: ', test_data_accuracy)

Accuracy on Test Data:  0.9354838709677419
